In [17]:
###############################################################
# 1. ENVIRONMENT SETUP
###############################################################

import os
from dotenv import load_dotenv
load_dotenv()


###############################################################
# 2. LANGGRAPH 0.6.10 IMPORTS
###############################################################

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver


###############################################################
# 3. LANGUAGE MODEL (LLAMA3 via OLLAMA)
###############################################################

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOllama(
    model="llama3",
    temperature=0.1
)


###############################################################
# 4. STATE DEFINITION
###############################################################

from typing import TypedDict, List

class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int


###############################################################
# 5. AIR-PACKED PROMPTS
###############################################################

PLAN_PROMPT = """
You are an expert essay planner.
Create a clean, clear outline for the user's topic.
Keep it structured, logical, and helpful.
"""

WRITER_PROMPT = """
You are an expert essay writer.
Write the best possible 5-paragraph essay using:

- The user's task
- The outline
- The research below

Research:
---------
{content}

If this is a revision, improve clarity, structure, depth, and correctness.
"""

REFLECTION_PROMPT = """
You are a strict teacher.
Provide direct critique of the essay:
- Missing depth
- Logic issues
- Style improvements
- Structural improvements
"""

RESEARCH_PLAN_PROMPT = """
Return EXACTLY 3 search queries as JSON ONLY:

{
  "queries": ["q1", "q2", "q3"]
}

Do not explain anything else.
"""

RESEARCH_CRITIQUE_PROMPT = """
Based on the critique, return EXACTLY 3 search queries as JSON ONLY:

{
  "queries": ["q1", "q2", "q3"]
}

Do not explain anything else.
"""


###############################################################
# 6. TAVILY SETUP
###############################################################

from tavily import TavilyClient
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


###############################################################
# 7. SAFE JSON PARSER FOR OLLAMA OUTPUT
###############################################################

import json

def extract_json(text):
    """
    Extracts the first valid JSON object found in the model output.
    If no JSON found, returns empty list.
    """
    try:
        start = text.index("{")
        end = text.rindex("}") + 1
        return json.loads(text[start:end])
    except:
        return {"queries": []}



###############################################################
# 8. NODE DEFINITIONS
###############################################################

############################
# NODE A — Planner
############################
def planner_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"])
    ]
    response = llm.invoke(messages)
    return {"plan": response.content}


############################
# NODE B — Research For Plan
############################
def research_plan_node(state: AgentState):

    # Ask LLaMA3 for JSON search queries
    prompt = RESEARCH_PLAN_PROMPT + f"\n\nTask: {state['task']}"
    response = llm.invoke([HumanMessage(content=prompt)])
    data = extract_json(response.content)

    content = state.get("content", [])

    # Run each query with Tavily
    for q in data.get("queries", []):
        results = tavily.search(query=q, max_results=2)
        for r in results["results"]:
            content.append(r["content"])

    return {"content": content}


############################
# NODE C — Essay Generation
############################
def generate_node(state: AgentState):

    research_text = "\n\n".join(state.get("content", []))

    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=research_text)),
        HumanMessage(content=f"Task: {state['task']}\n\nOutline:\n{state['plan']}")
    ]

    response = llm.invoke(messages)

    return {
        "draft": response.content,
        "revision_number": state["revision_number"] + 1
    }


############################
# NODE D — Reflection
############################
def reflect_node(state: AgentState):

    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state["draft"])
    ]

    response = llm.invoke(messages)
    return {"critique": response.content}


############################
# NODE E — Research for Critique
############################
def research_critique_node(state: AgentState):

    prompt = RESEARCH_CRITIQUE_PROMPT + f"\n\nCritique: {state['critique']}"
    response = llm.invoke([HumanMessage(content=prompt)])
    data = extract_json(response.content)

    content = state.get("content", [])

    for q in data.get("queries", []):
        results = tavily.search(query=q, max_results=2)
        for r in results["results"]:
            content.append(r["content"])

    return {"content": content}



###############################################################
# 9. CONDITIONAL ROUTING
###############################################################

def should_continue(state: AgentState):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"



###############################################################
# 10. BUILD THE GRAPH
###############################################################

checkpointer = MemorySaver()
builder = StateGraph(AgentState)

builder.add_node("planner", planner_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("generate", generate_node)
builder.add_node("reflect", reflect_node)
builder.add_node("research_critique", research_critique_node)

# Entry
builder.set_entry_point("planner")

# Normal flow
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

# Conditional rewrite loop
builder.add_conditional_edges(
    "generate",
    should_continue,
    {
        END: END,
        "reflect": "reflect"
    }
)

# Compile
graph = builder.compile(checkpointer=checkpointer)



###############################################################
# 11. RUN THE GRAPH
###############################################################

thread = {"configurable": {"thread_id": "essay_run_1"}}

print("\n=== STREAM OUTPUT ===\n")

for step in graph.stream(
    {
        "task": "What is the difference between LangChain and LangSmith?",
        "max_revisions": 2,
        "revision_number": 1
    },
    thread
):
    print(step)



###############################################################
# 12. GUI SUPPORT (OPTIONAL)
###############################################################
# from helper import ewriter, writer_gui
# MultiAgent = ewriter()
# app = writer_gui(MultiAgent.graph)
# app.launch()



=== STREAM OUTPUT ===

{'planner': {'plan': "Here is a suggested outline for your essay on the differences between LangChain and LangSmith:\n\nI. Introduction\n\n* Brief overview of LangChain and LangSmith\n* Thesis statement: While both LangChain and LangSmith are AI-powered language processing tools, they differ in their approaches, capabilities, and applications.\n\nII. Background Information\n\n* Definition of LangChain: [insert brief description]\n* Definition of LangSmith: [insert brief description]\n\nIII. Key Differences\n\n* A. Approach:\n\t+ LangChain: [briefly describe LangChain's approach to language processing]\n\t+ LangSmith: [briefly describe LangSmith's approach to language processing]\n* B. Capabilities:\n\t+ LangChain: [list specific capabilities of LangChain, e.g., text generation, summarization, etc.]\n\t+ LangSmith: [list specific capabilities of LangSmith, e.g., text analysis, sentiment detection, etc.]\n* C. Applications:\n\t+ LangChain: [discuss potential appli